In [1]:
pip install pdf2image

In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pdf2image import convert_from_path
import os

# Chemin vers le dossier contenant les fichiers PDF
dossier_pdf = "C:\\Users\\berra\\Desktop\\Stage\\extrait_suspects"

# Liste des chemins complets des fichiers PDF dans le dossier
chemins_pdf = [os.path.join(dossier_pdf, nom_fichier) for nom_fichier in os.listdir(dossier_pdf) if nom_fichier.endswith('.pdf')]

# Créer un dossier pour stocker les images extraites
dossier_images = "C:\\Users\\berra\\Desktop\\Stage\\images_extraits_suspects"
os.makedirs(dossier_images, exist_ok=True)

# Parcourir les fichiers PDF
for chemin_pdf in chemins_pdf:
    # Convertir le PDF en une liste d'images
    images = convert_from_path(chemin_pdf)

    # Parcourir les images
    for i, image in enumerate(images):
        # Sauvegarder chaque image dans le dossier des images extraites
        nom_image = f"{os.path.splitext(os.path.basename(chemin_pdf))[0]}_{i+1}.jpg"
        chemin_image = os.path.join(dossier_images, nom_image)
        image.save(chemin_image, "JPEG")
        print(f"Image {i+1} du fichier {chemin_pdf} sauvegardée sous {chemin_image}")

Image 1 du fichier C:\Users\berra\Desktop\Stage\extrait_suspects\birth_act (1).pdf sauvegardée sous C:\Users\berra\Desktop\Stage\images_extraits_suspects\birth_act (1)_1.jpg
Image 1 du fichier C:\Users\berra\Desktop\Stage\extrait_suspects\birth_act (2).pdf sauvegardée sous C:\Users\berra\Desktop\Stage\images_extraits_suspects\birth_act (2)_1.jpg
Image 1 du fichier C:\Users\berra\Desktop\Stage\extrait_suspects\birth_act (3).pdf sauvegardée sous C:\Users\berra\Desktop\Stage\images_extraits_suspects\birth_act (3)_1.jpg
Image 1 du fichier C:\Users\berra\Desktop\Stage\extrait_suspects\birth_act (4).pdf sauvegardée sous C:\Users\berra\Desktop\Stage\images_extraits_suspects\birth_act (4)_1.jpg
Image 1 du fichier C:\Users\berra\Desktop\Stage\extrait_suspects\birth_act (5).pdf sauvegardée sous C:\Users\berra\Desktop\Stage\images_extraits_suspects\birth_act (5)_1.jpg
Image 1 du fichier C:\Users\berra\Desktop\Stage\extrait_suspects\birth_act (6).pdf sauvegardée sous C:\Users\berra\Desktop\Stage\i

In [14]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans

def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

def redimensionner_caracteristiques(caracteristiques, longueur_cible):
    if caracteristiques.shape[0] < longueur_cible:
        caracteristiques = np.concatenate([caracteristiques, np.zeros((longueur_cible - caracteristiques.shape[0], 128))])
    elif caracteristiques.shape[0] > longueur_cible:
        caracteristiques = caracteristiques[:longueur_cible]
    return caracteristiques

# Définir une longueur cible pour les caractéristiques
longueur_cible = 500

# Initialiser le modèle KMeans
kmeans = KMeans(n_clusters=2)

# Chemin vers les fichiers des images extraites
chemin_images_cachets_suspects = "C:\\Users\\berra\\Desktop\\Stage\\images_extraits_suspects"
chemin_images_signatures_suspects = "C:\\Users\\berra\\Desktop\\Stage\\images_extraits_suspects"

# Liste des caractéristiques des cachets suspects
caracteristiques_cachets_suspects = []

# Liste des caractéristiques des signatures suspects
caracteristiques_signatures_suspects = []

# Parcourir les images des cachets suspects
for fichier in os.listdir(chemin_images_cachets_suspects):
    chemin_image = os.path.join(chemin_images_cachets_suspects, fichier)
    image = cv2.imread(chemin_image)
    caracteristiques = extract_features(image)
    caracteristiques = redimensionner_caracteristiques(caracteristiques, longueur_cible)
    caracteristiques_cachets_suspects.append(caracteristiques.flatten())  

# Parcourir les images des signatures suspects
for fichier in os.listdir(chemin_images_signatures_suspects):
    chemin_image = os.path.join(chemin_images_signatures_suspects, fichier)
    image = cv2.imread(chemin_image)
    caracteristiques = extract_features(image)
    caracteristiques = redimensionner_caracteristiques(caracteristiques, longueur_cible)
    caracteristiques_signatures_suspects.append(caracteristiques.flatten())  

# Convertir les listes en tableaux numpy
caracteristiques_cachets_suspects = np.array(caracteristiques_cachets_suspects)
caracteristiques_signatures_suspects = np.array(caracteristiques_signatures_suspects)

# Combinez les caractéristiques de cachets et de signatures en un seul tableau pour chaque catégorie
caracteristiques_combines_cachets_signatures = np.concatenate((caracteristiques_cachets_suspects, caracteristiques_signatures_suspects), axis=0)

# Entraîner le modèle KMeans sur l'ensemble combiné de caractéristiques
kmeans.fit(caracteristiques_combines_cachets_signatures)

# Prédire les clusters pour les cachets et les signatures
etiquettes_clusters_cachets = kmeans.predict(caracteristiques_cachets_suspects)
etiquettes_clusters_signatures = kmeans.predict(caracteristiques_signatures_suspects)

# Afficher les résultats pour les cachets
print("Résultats pour les cachets :")
for i, fichier in enumerate(os.listdir(chemin_images_cachets_suspects)):
    if etiquettes_clusters_cachets[i] == 0:
        print(f"Le cachet dans {fichier} est authentique.")
    else:
        print(f"Le cachet dans {fichier} est suspect.")

# Afficher les résultats pour les signatures
print("\nRésultats pour les signatures :")
for i, fichier in enumerate(os.listdir(chemin_images_signatures_suspects)):
    if etiquettes_clusters_signatures[i] == 0:
        print(f"La signature dans {fichier} est authentique.")
    else:
        print(f"La signature dans {fichier} est suspecte.")

Résultats pour les cachets :
Le cachet dans birth_act (1)_1.jpg est authentique.
Le cachet dans birth_act (2)_1.jpg est authentique.
Le cachet dans birth_act (2)_11.jpg est suspect.
Le cachet dans birth_act (3)_1.jpg est authentique.
Le cachet dans birth_act (4)_1.jpg est authentique.
Le cachet dans birth_act (5)_1.jpg est suspect.
Le cachet dans birth_act (5)_11.jpg est suspect.
Le cachet dans birth_act (6)_1.jpg est suspect.
Le cachet dans birth_act (7)_1.jpg est authentique.
Le cachet dans birth_act (8)_1.jpg est authentique.
Le cachet dans birth_act (9)_1.jpg est authentique.
Le cachet dans birth_act (9)_11.jpg est authentique.

Résultats pour les signatures :
La signature dans birth_act (1)_1.jpg est authentique.
La signature dans birth_act (2)_1.jpg est authentique.
La signature dans birth_act (2)_11.jpg est suspecte.
La signature dans birth_act (3)_1.jpg est authentique.
La signature dans birth_act (4)_1.jpg est authentique.
La signature dans birth_act (5)_1.jpg est suspecte.
La